# High-level datasets

{py:class}`xarray_beam.Dataset` is new (as of September 2025) high-level interface for Xarray-Beam.

It requires less boilerplate code than the current (explicit) interface, and accordingly should be an easier to use tool, especially for non-expert users. You still need to think about how your data is divided into chunks, but the data model of `Dataset` keep track of the high-level structure of your data, avoiding the need to manually building templates in {py:class}`~xarray_beam.ChunksToZarr`.

```{warning}
The `Dataset` interface is experimental, and currently offers no backwards compatibility guarantees.
```

## Data model

Dataset is a wrapper over a series of Beam transformations, adding metadata describing the corresponding `xarray.Dataset` and how is distributed with Beam:

- `ptransform` is the wrapped `beam.PTransform` to compute the chunks of the dataset.
- `template` is a lazily-computed `xarray.Dataset` indicating the structure of the overall dataset.
- `chunks` is dictionary mapping from dimension names to integer chunk sizes, indicating the size of each chunk.
- `split_vars` is a boolean indicating if ptransform elements each contain only a single variable from the dataset, rather than all variables.

This information is surfaced via `xbeam.Dataset.__repr__()`:

In [1]:
import apache_beam as beam
import xarray_beam as xbeam
import xarray
import numpy as np
import pandas as pd

xarray_ds = xarray.Dataset(
    {'temperature': (('time', 'longitude', 'latitude'), np.random.randn(365, 360, 180))},
    coords={'time': pd.date_range('2025-01-01', freq='1D', periods=365)},
)
chunks = {'time': '30MB', 'longitude': -1, 'latitude': -1}
xbeam_ds = xbeam.Dataset.from_xarray(xarray_ds, chunks)
xbeam_ds

Xarray-Beam pipelines typically read and write data to [Zarr](https://zarr.dev/), so we'll start by writing our example data to a Zarr file (with plain Xarray/Dask) for us to read later:

In [2]:
xarray_ds.chunk(chunks).to_zarr('example_data.zarr', mode='w')

## Writing pipelines

Most Xarray-Beam pipelines can be written via a handful of Dataset methods:

- {py:meth}`~xarray_beam.Dataset.from_zarr`: Load a dataset from a Zarr store.
- {py:meth}`~xarray_beam.Dataset.rechunk`: Adjust chunks on a dataset.
- {py:meth}`~xarray_beam.Dataset.map_blocks`: Map a function over every chunk of this dataset independently.
- {py:meth}`~xarray_beam.Dataset.to_zarr`: Write a dataset to a Zarr store.

All non-trivial computation happens via the embarrasingly parallel `map_blocks` method.

### Chunking strategies

In order for `map_blocks` to work, data needs to be appropriately chunked. Here are a few typical chunking patterns that work well for most needs:

- "Pencil" chunks, which group together all times, and parallelize over space. These long and skinny chunks look like a box of pencils.
- "Pancake" chunks, which group together all spatial locations, and parallelize over time. These flat and wide chunks look like a stack of pancakes.
- Intermediate "compromise" chunks, somewhere between these extremes. These optimize for worst-case behavior. 

![Pancake vs Pencil chunks](./_static/pancake-vs-pencil.svg)


Weather/climate datasets are typically generated and stored in "pancake" chunks, but "pencil" chunks are more suitable for most analytics queries, which requires large histories of weather at small numbers of locations.

Using the right chunks is *absolutely essentially* for efficient operations with Xarray-Beam and Zarr. For example, reading data from a single location across all times (a "pencil" query) is extremely inefficient for a dataset stored in "pancake" chunks -- it would require loading the entire dataset from disk!

[Like Dask](https://docs.dask.org/en/latest/array-best-practices.html), Xarray-Beam works best with chunks that are 10s to 100s of MB in size, large enough to amortize Python overhead but small enough that chunks fit easily into memory. Smaller chunk sizes in Zarr stores (closer to 10MB) can be convenient to increase the flexiblity of chunking methods when reading the data from disk later

You can explicitly adjusts chunk sizes with {py:meth}`~xarray_beam.Dataset.rechunk`. The syntax is a mapping from dimension names (or `...`, for all other dimensions) to chunk sizes, which can be any of `-1` (to indicate "size of the full dimension"), a positive integer or a string indicating a target size in bytes like `'100MB'`, e.g.,
- `chunks=-1`: no chunking.
- `chunks='100 MB'`: chunk size of ~100 MB, without dimension-specific constraints.
- `chunks={'time': 100}`: fixed size of 100 along time, preserving original chunks along other dimensions.
- `chunks={'time': -1, ...: '100 MB'}`: pencil chunks of ~100 MB each.
- `chunks={'latitude': -1, 'longitude': -1, ...: '100 MB'}`: pancake chunks of ~100 MB each.

See {py:func}`~xarray_beam.normalize_chunks` for full details.


```{warning}
Rechunking between very different schemes is fundamentally an expensive operation (it requires multiple complete reads/writes of a dataset from disk). If performance and flexibility are critical, it may be worth storing multiple copies of your data in different chunking formats.
```

```{tip}
Using Zarr v3's sharding feature in {py:meth}`~xarray_beam.Dataset.to_zarr` to group smaller chunks (~1 MB) into shards can also help mitigate the challenges of picking an optimal chunk size.
```

### Example 1: Climatology

Here we need to group together all time points in the same chunk ("pencil chunks"), parallelizing over space:

In [3]:
with beam.Pipeline() as p:
  p | (
      xbeam.Dataset.from_zarr('example_data.zarr')
      .rechunk({'time': -1, ...: '100 MB'})
      .map_blocks(lambda ds: ds.groupby('time.month').mean())
      .rechunk('10 MB')  # ensure a reasonable min chunk-size for Zarr
      .to_zarr('example_climatology.zarr')
  )
xarray.open_zarr('example_climatology.zarr')

### Example 2: Regridding over space

Here we need to group all space points in the same chunk ("pancake chunks"), parallelizing over time:

In [4]:
with beam.Pipeline() as p:
  p | (
      xbeam.Dataset.from_zarr('example_data.zarr')
      .rechunk({'time': '30MB', 'latitude': -1, 'longitude': -1})
      .map_blocks(lambda ds: ds.coarsen(latitude=2, longitude=2).mean())
      .to_zarr('example_regrid.zarr')
  )
xarray.open_zarr('example_regrid.zarr')

## Limitations of map_blocks

In the examples above, {py:meth}`~xarray_beam.Dataset.map_blocks` somehow automatically knew the appropriate structure of the output `template`, without evaluating any chunked data. How could this work?

For building templates, Xarray-Beam relies on lazy evaluation with [Dask arrays](https://docs.dask.org/en/stable/array.html). This requires that applied functions are Dask compatible. Almost all built-in Xarray operations are Dask compatible, but if your applied function is _not_ Dask compatible (e.g., because it loads array values into memory), Xarray-Beam will show an informative error:

In [5]:
try:
  (
      xbeam.Dataset.from_zarr('example_data.zarr')
      .map_blocks(lambda ds: ds.compute())  # load into memory
  )
except Exception as e:
  print(f'{type(e).__name__}: {e}')

You can avoid these errors by explicitly [creating a template](creating_templates):

In [6]:
(
    xbeam.Dataset.from_zarr('example_data.zarr')
    .map_blocks(lambda ds: ds.compute(), template=ds_beam.template)
)

## Interfacing with low-level transforms

`Dataset` is a thin wrapper around Xarray-Beam transformations, so you can always drop into the lower-level Xarray-Beam [data model](data-model) and use raw Beam operations. This is especially useful for the reading or writing data.

```{warning}
The `Dataset` constructor currently performs **no validation** on its inputs!
```

For example, here's how you could manually recreate a `Dataset`, using the common pattern of evaluating a single example in-memory to create a template with {py:func}`~xarray_beam.make_template` and {py:func}`~xarray_beam.replace_template_dims`:

In [12]:
all_times = pd.date_range('2025-01-01', freq='1D', periods=365)
source_dataset = xarray.open_zarr('example_data.zarr', chunks=None)

def load_chunk(time: pd.Timestamp) -> tuple[xbeam.Key, xarray.Dataset]:
  key = xbeam.Key({'time': (time - all_times[0]).days})
  dataset = source_dataset.sel(time=[time])
  return key, dataset

_, example = load_chunk(all_times[0])

template = xbeam.make_template(example)
template = xbeam.replace_template_dims(template, time=all_times)

ds_beam = xbeam.Dataset(
    template=template,
    chunks=xbeam.normalize_chunks({'time': 1}, template),
    split_vars=False,
    ptransform=(beam.Create(all_times) | beam.Map(load_chunk)),
)
ds_beam

You can also pull-out the underlying Beam `ptransform` from a dataset to append new transformations, e.g., to write each element of the pipeline to disk as a separate file:

In [13]:
def to_netcdf(key: xbeam.Key, chunk: xarray.Dataset):
  path = f"{chunk.indexes['time'][0]:%Y-%m-%d}.nc"
  chunk.to_netcdf(path)

with beam.Pipeline() as p:
  p | ds_beam.rechunk('50MB').ptransform | beam.MapTuple(to_netcdf)

%ls *.nc